# Biblioteca de Algoritmos - Lab 03

Nos últimos anos, muitas bibliotecas RL foram desenvolvidas. Essas bibliotecas foram projetadas para ter todas as ferramentas necessárias para implementar e testar agentes de Aprendizado por Reforço .

Ainda assim, elas se diferem muito. É por isso que é importante escolher uma biblioteca que seja rápida, confiável e relevante para sua tarefa de RL. Do ponto de vista técnico, existem algumas coisas a se ter em mente ao considerar uma bilioteca para RL.

- **Suporte para bibliotecas de aprendizado de máquina existentes:** Como o RL normalmente usa algoritmos baseados em gradiente para aprender e ajustar funções de política, você vai querer que ele suporte sua biblioteca favorita (Tensorflow, Keras, Pytorch, etc.)
- **Escalabilidade:** RL é computacionalmente intensivo e ter a opção de executar de forma distribuída torna-se importante ao atacar ambientes complexos.
- **Composibilidade:** Os algoritmos de RL normalmente envolvem simulações e muitos outros componentes. Você vai querer uma biblioteca que permita reutilizar componentes de algoritmos de RL, que seja compatível com várias estruturas de aprendizado profundo.

[Aqui](https://docs.google.com/spreadsheets/d/1ZWhViAwCpRqupA5E_xFHSaBaaBZ1wAjO6PvmmEEpXGI/edit#gid=0) você consegue visualizar uma lista com algumas bibliotecas existentes.

<img src="https://i1.wp.com/neptune.ai/wp-content/uploads/RL-tools.png?resize=1024%2C372&ssl=1" width=500>


## Ray RLlib

[Ray](https://docs.ray.io/en/latest/) é uma plataforma de execução distribuída que fornece bases para paralelismo e escalabilidade que são simples de usar e permitem que os programas Python sejam escalados em qualquer lugar, de um notebook a um grande cluster. Além disso, construída sobre o Ray, temos a [RLlib](https://docs.ray.io/en/latest/rllib.html), que fornece uma API unificada que pode ser aproveitada em uma ampla gama de aplicações.

<br>

<img src="https://miro.medium.com/max/1838/1*_bomm09XtiZfQ52Kfz9Ciw.png" width=600>


A RLlib foi projetada para oferecer suporte a várias estruturas de aprendizado profundo (TensorFlow e PyTorch) e pode ser acessada por meio de uma API Python simples. Atualmente, ela vem com uma [série de algoritmos RL](https://docs.ray.io/en/latest/rllib-algorithms.html#available-algorithms-overview).

Em particular, a RLlib permite um desenvolvimento rápido porque torna mais fácil construir algoritmos RL escaláveis ​​por meio da reutilização e montagem de implementações existentes. A RLlib também permite que os desenvolvedores usem redes neurais criadas com várias estruturas de aprendizado profundo e se integra facilmente a simuladores de terceiros.


## Configuração

Você precisará fazer uma cópia deste notebook em seu Google Drive antes de editar. Você pode fazer isso com **Arquivo → Salvar uma cópia no Drive**.

In [ ]:
import os
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
# Seu trabalho será armazenado em uma pasta chamada `minicurso_rl` por padrão 
# para evitar que o tempo limite da instância do Colab exclua suas edições

DRIVE_PATH = "/content/gdrive/MyDrive/minicurso_rl/lab03"
DRIVE_PYTHON_PATH = DRIVE_PATH.replace("\\", "")
if not os.path.exists(DRIVE_PYTHON_PATH):
  %mkdir -p $DRIVE_PATH

In [ ]:
# a versão do ray compatível com a implementação dos agentes disponibilizada é a 1.4.0
!pip install 'aioredis==1.3.1' > /dev/null 2>&1 
!pip install 'ray==1.4.0' > /dev/null 2>&1 
!pip install 'ray[rllib]==1.4.0' > /dev/null 2>&1 
!pip install 'ray[tune]==1.4.0' > /dev/null 2>&1 
!pip install torch > /dev/null 2>&1 
!pip install lz4 > /dev/null 2>&1 

# Dependências necessárias para gravar os vídeos
!apt-get install -y xvfb x11-utils > /dev/null 2>&1 
!pip install pyvirtualdisplay==0.2.* > /dev/null 2>&1 

# Ambiente da competição
!pip install --upgrade ceia-soccer-twos > /dev/null 2>&1

In [ ]:
! wget http://www.atarimania.com/roms/Roms.rar
! mkdir /content/ROM/
! unrar e /content/Roms.rar /content/ROM/ -y
! python -m atari_py.import_roms /content/ROM/ > /dev/null 2>&1

In [ ]:
# Inicializa uma instância de um display virtual
from pyvirtualdisplay import Display
display = Display(visible=False, size=(1400, 900))
_ = display.start()

In [ ]:
# Carrega a extensão do notebook TensorBoard
%load_ext tensorboard

## Ambiente

O OpenAI Gym possui um wrapper VideoRecorder que pode gravar um vídeo do ambiente em formato MP4. Abaixo iremos interagir no ambiente do [Carpole](https://gym.openai.com/envs/CartPole-v0/) executando ações aleatórias e gravar o resultado.

In [ ]:
import gym
from gym.wrappers.monitoring.video_recorder import VideoRecorder

environment_id = "CartPole-v0"

In [ ]:
import gym
from gym.wrappers.monitoring.video_recorder import VideoRecorder

env = gym.make(environment_id)
before_training = os.path.join(
    DRIVE_PATH, "{}_before_training.mp4".format(environment_id)
)
print(before_training)

video = VideoRecorder(env, before_training)
env.reset()
for i in range(200):
  env.render()
  video.capture_frame()
  observation, reward, done, info = env.step(env.action_space.sample())

video.close()
env.close()

O código acima salvou o arquivo de vídeo no seu Drive. Para exibi-lo no notebook, você precisa de uma função auxiliar.

In [ ]:
from base64 import b64encode
def render_mp4(videopath: str) -> str:
  mp4 = open(videopath, 'rb').read()
  base64_encoded_mp4 = b64encode(mp4).decode()
  return f'<video width=400 controls><source src="data:video/mp4;' \
         f'base64,{base64_encoded_mp4}" type="video/mp4"></video>'

O código abaixo renderiza os resultados. Você deve obter um vídeo semelhante ao abaixo.

In [ ]:
from IPython.display import HTML
html = render_mp4(before_training)
HTML(html)

## Treinando um agente de Aprendizado por Reforço

Primeiro, vamos começar a executar o Ray em segundo plano. Executar um `ray.shutdown()` seguido por um `ray.init()` deve dar início às coisas.

In [ ]:
import ray

ray.shutdown()
ray.init(ignore_reinit_error=True, include_dashboard=False)

### Basic Python API

Em alto nível, RLlib fornece uma classe Trainer que contém uma política para interação com o ambiente. Por meio da interface do Trainer, a política pode ser treinada, avaliada ou computar uma ação. 

Para cada algoritmo gostaríamos de configurar os parâmetros (taxa de aprendizado, tamanho da rede, tamanho do batch, etc.) de acordo com a nossa aplicação.  Para isso o Ray fornece dois níveis de paramêtros que podemos alterar. Primeiramente temos os parâmetros comuns a todos os algoritmos. Você pode conferir uma lista com os parâmetros disponíveis através desse [link](https://docs.ray.io/en/latest/rllib-training.html#common-parameters).

E para cada [algoritmo disponível no ray](https://docs.ray.io/en/latest/rllib-algorithms.html#available-algorithms-overview) temos os parâmetros específicos. Na imagem abaixo podemos ver os parâmetros específicos para o algoritmo [Policy Gradient](https://docs.ray.io/en/latest/rllib-algorithms.html#policy-gradients).


<img src='https://drive.google.com/uc?id=1yKJDJViHE_F9JH7NTQMYtQL3KLBJoJyk' width="500" >


In [ ]:
import ray
import ray.rllib.agents.pg as pg
from ray.tune.logger import pretty_print

config = pg.DEFAULT_CONFIG.copy()
config["num_gpus"] = 0
config["num_workers"] = 1
config["lr"] = 0.0004
config["framework"] = "torch"

trainer = pg.PGTrainer(config=config, env=environment_id)
episodes = 1000

for i in range(episodes):
   # Executa uma iteração de treinamento da política com Policy Gradient (PG)
   result = trainer.train()
   print(pretty_print(result))

   if i % 100 == 0:
       checkpoint = trainer.save()
       print("checkpoint saved at", checkpoint)

last_checkpoint = trainer.save()

In [ ]:
print("Last checkpoint saved at", last_checkpoint)

Agora vamos criar outro vídeo, mas desta vez escolha a ação recomendada pelo modelo treinado em vez de agir aleatoriamente.

In [ ]:
trainer = pg.PGTrainer(config=config, env=environment_id)
trainer.restore(last_checkpoint)

after_training = os.path.join(
    DRIVE_PATH, "{}after_training_basic_api.mp4".format(environment_id)
)
after_video = VideoRecorder(env, after_training)
observation = env.reset()
done = False
while not done:
  env.render()
  after_video.capture_frame()
  action = trainer.compute_action(observation)
  observation, reward, done, info = env.step(action)
after_video.close()
env.close()
html = render_mp4(after_training)
HTML(html)

### Usando ambiente ou modelos personalizados

A API Python fornece a flexibilidade necessária para aplicar o RLlib a novos problemas. Você precisará usar esta API se desejar usar ambientes ou modelos personalizados com RLlib. Abaixo veremos um exemplo de um ambiente e um modelo customizado.

<br>


Para maiores informações veja em [APIs Python avançadas](https://docs.ray.io/en/latest/rllib-training.html#advanced-python-apis).

In [ ]:
import gym
from gym.spaces import Discrete, Box
import numpy as np
import os
import random

import torch
import torch.nn as nn

import ray
from ray import tune
from ray.rllib.agents import pg
from ray.rllib.env.env_context import EnvContext
from ray.rllib.models import ModelCatalog
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.models.torch.fcnet import FullyConnectedNetwork as TorchFC
from ray.tune.logger import pretty_print

In [ ]:
class SimpleCorridor(gym.Env):
    """Exemplo de um ambiente personalizado em que você tem que andar por um 
    corredor. Você pode configurar o comprimento do corredor através da 
    configuração do ambiente."""

    def __init__(self, config: EnvContext):
        self.end_pos = config["corridor_length"]
        self.cur_pos = 0
        self.action_space = Discrete(2)
        self.observation_space = Box(
            0.0, self.end_pos, shape=(1, ), dtype=np.float32)
        # Define a seed. É usado apenas para a recompensa final.
        self.seed(config.worker_index * config.num_workers)

    def reset(self):
        self.cur_pos = 0
        return [self.cur_pos]

    def step(self, action):
        assert action in [0, 1], action
        if action == 0 and self.cur_pos > 0:
            self.cur_pos -= 1
        elif action == 1:
            self.cur_pos += 1
        done = self.cur_pos >= self.end_pos
        # Produz uma recompensa aleatória quando atingirmos a meta.
        return [self.cur_pos], \
            random.random() * 2 if done else -0.1, done, {}

    def seed(self, seed=None):
        random.seed(seed)

In [ ]:
class TorchCustomModel(TorchModelV2, nn.Module):
    """Exemplo de um modelo personalizado PyTorch que apenas delega para uma 
    fc-net."""

    def __init__(self, obs_space, action_space, num_outputs, model_config,
                 name):
        TorchModelV2.__init__(self, obs_space, action_space, num_outputs,
                              model_config, name)
        nn.Module.__init__(self)

        self.torch_sub_model = TorchFC(obs_space, action_space, num_outputs,
                                       model_config, name)

    def forward(self, input_dict, state, seq_lens):
        input_dict["obs"] = input_dict["obs"].float()
        fc_out, _ = self.torch_sub_model(input_dict, state, seq_lens)
        return fc_out, []

    def value_function(self):
        return torch.reshape(self.torch_sub_model.value_function(), [-1])

In [ ]:
# Também pode registrar a função de criar um ambiente explicitamente com:
# register_env("corridor", lambda config: SimpleCorridor(config))

# Registrar o modelo customizado
ModelCatalog.register_custom_model(
    "my_model", TorchCustomModel
)

config = {
    "env": SimpleCorridor,  # ou "corridor" se registrado
    "env_config": {
        "corridor_length": 5,
    },
    "model": {
        "custom_model": "my_model",
        "vf_share_layers": True,
    },
    "num_workers": 1,  
    "framework": "torch",
}

stop = {
    "training_iteration": 50,
    "timesteps_total": 100000,
    "episode_reward_mean": 0.1,
}

In [ ]:
pg_config = pg.DEFAULT_CONFIG.copy()
pg_config.update(config)
pg_config["lr"] = 1e-3

trainer = pg.PGTrainer(config=pg_config, env=SimpleCorridor)
# executa o loop de treinamento manual e imprime os resultados após cada iteração
for _ in range(stop["training_iteration"]):
    result = trainer.train()
    print(pretty_print(result))
    
    # pare o treinamento caso tiver alcançado a quantidade de steps desejada
    # ou caso a recompensa desejada seja alcançada
    if result["timesteps_total"] >= stop["timesteps_total"] or \
            result["episode_reward_mean"] >= stop["episode_reward_mean"]:
        break

### Ray Tune

Todos os Trainers do RLlib são compatíveis com a API do [Ray Tune](https://docs.ray.io/en/master/tune/index.html). Isso permite que eles sejam facilmente usados em experimentos com o Tune. Por exemplo, o código a seguir executa o mesmo treino com o CartPole com o algoritmo PG.

In [ ]:
import ray
config = {
    "env": environment_id,
    "framework": "torch",
}
stop = {"episode_reward_mean": 150, "timesteps_total": 100000}

# Executar o treinamento
analysis = ray.tune.run(
    "PG",
    config=config,
    stop=stop,
    checkpoint_freq=10,
    checkpoint_at_end=True,
    local_dir=os.path.join(DRIVE_PATH, "results")
)

Embora o objeto de análise retornado do `ray.tune.run` anteriormente não tivesse nenhuma instância Trainer, ele tem todas as informações necessárias para reconstruir um de um checkpoint salvo.

O retorno do Ray Tune é um objeto [ExperimentAnalysis](https://docs.ray.io/en/latest/tune/api_docs/analysis.html?highlight=ExperimentAnalysis#experimentanalysis-tune-experimentanalysis) onde é possível resgatar qual o melhor checkpoint do treino.

In [ ]:
from ray.rllib.agents.pg import PGTrainer

# restaurar um Trainer 
trial = analysis.get_best_logdir("episode_reward_mean", "max")
checkpoint = analysis.get_best_checkpoint(
  trial,
  "training_iteration",
  "max",
)
trainer = PGTrainer(config=config)
trainer.restore(checkpoint)

Agora vamos criar outro vídeo, mas desta vez escolha a ação recomendada pelo modelo treinado com a API Tune.

In [ ]:
after_training = after_training = os.path.join(
    DRIVE_PATH, "{}after_training_tune.mp4".format(environment_id)
)
after_video = VideoRecorder(env, after_training)
observation = env.reset()
done = False
while not done:
  env.render()
  after_video.capture_frame()
  action = trainer.compute_action(observation)
  observation, reward, done, info = env.step(action)
after_video.close()
env.close()
# You should get a video similar to the one below. 
html = render_mp4(after_training)
HTML(html)

O Tune gera arquivos do [Tensorboard](https://www.tensorflow.org/tensorboard) automaticamente durante o `tune.run()` Para visualizar a aprendizagem no tensorboard, execute o célula abaixo:

In [ ]:
%tensorboard --logdir /content/gdrive/MyDrive/minicurso_rl/lab03/results/PG

## Hyperparameter Tuning com o Ray Tune

[Ray Tune](https://docs.ray.io/en/latest/tune/index.html) é uma biblioteca para execução de experimentos e ajuste de hiperparâmetros. Vamos agora tentar encontrar hiperparâmetros que possam resolver o ambiente [Cartpole](https://gym.openai.com/envs/CartPole-v1/) no menor número de passos de tempo. Esteja preparado para que demore um pouco para ser executado.

In [ ]:
parameter_search_config = {
    "env": environment_id,
    "framework": "torch",
    "num_gpus": 1,  # porcentagem da gpu disponível para treino
    "num_workers": 1,  # número de workers além do processo principal; no colab deve ser 1 pois só há 2 CPUs

    # Hyperparameter tuning
    "model": {
      "fcnet_hiddens": ray.tune.grid_search([[32], [64]]),
      "fcnet_activation": ray.tune.grid_search(["linear", "relu"]),
    },
    "lr": ray.tune.uniform(1e-7, 1e-2)
}

# To explicitly stop or restart Ray, use the shutdown API.
ray.shutdown()

ray.init(
  num_cpus=2,
  include_dashboard=False,
  ignore_reinit_error=True,
  log_to_driver=False,
)

parameter_search_analysis = ray.tune.run(
  "PG",
  config=parameter_search_config,
  stop=stop,
  num_samples=5,
  metric="timesteps_total",
  mode="min",
)

In [ ]:
print(
  "Melhores hiperparâmetros encontrados:",
  parameter_search_analysis.best_config,
)

Especificando num_samples = 5 significa que você obterá cinco amostras aleatórias para a taxa de aprendizagem. Para cada um deles, existem dois valores para o tamanho da camada oculta e dois valores para a função de ativação. Portanto, haverá 5 * 2 * 2 = 20 tentativas, mostradas com seus status na saída da célula à medida que o cálculo é executado.

Observe que Ray mostra a melhor configuração atual à medida que avança. Isso inclui todos os valores padrão que foram definidos, o que é um bom lugar para encontrar outros parâmetros que podem ser ajustados.


## Exercício

Agora que você conhece a API básica do Ray Tune e da RLLib, **utilize o ambiente `BreakoutNoFrameskip-v4` e treine agentes com os algoritmos A3C, PPO e SAC**. Lembre-se de utilizar também o tensorboard para acompanhar e comparar as curvas de aprendizado de suas execuções.

Descrições dos algoritmos e seus respectivos hiperparâmetros podem ser encontrados [aqui](https://docs.ray.io/en/latest/rllib-algorithms.html#available-algorithms-overview).

In [ ]:
# INSIRA AQUI O CÓDIGO PARA TREINAMENTO SOBRE O BreakoutNoFrameskip-v4

# Bônus

Como tarefa bônus, experimente com os algoritmos aprendidos no ambiente `soccer_twos`, que será utilizado na competição final deste curso*. Para facilitar, utilize a variação `team_vs_policy` como no laboratório anterior.

<img src="https://raw.githubusercontent.com/bryanoliveira/soccer-twos-env/master/images/screenshot.png" height="400">

> Visualização do ambiente

Este ambiente consiste em um jogo de futebol de carros 2x2, ou seja, o objetivo é marcar um gol no adversário o mais rápido possível. Na variação `team_vs_policy`, seu agente controla um jogador do time azul e joga contra um time aleatório. Mais informações sobre o ambiente podem ser encontradas [no repositório](https://github.com/bryanoliveira/soccer-twos-env) e [na documentação do Unity ml-agents](https://github.com/Unity-Technologies/ml-agents/blob/main/docs/Learning-Environment-Examples.md#soccer-twos).


**Sua tarefa é treinar um agente com a interface do Ray apresentada, experimentando com diferentes algoritmos e hiperparâmetros.**


<br>

*A variação utilizada na competição será a `multiagent_player`, mas agentes treinados para `team_vs_policy` podem ser facilmente adaptados. Na seção "Exportando seu agente treinado" o agente "MyDqnSoccerAgent" faz exatamente isso.

Utilize o ambiente instanciado abaixo para executar o algoritmo de treinamento. Ao final da execução, a recompensa do seu agente por episódio deve tender a +2.

In [ ]:
import soccer_twos

# Fecha o ambiente caso tenha sido aberto anteriormente
try: env.close()
except: pass

env = soccer_twos.make(
    variation=soccer_twos.EnvType.team_vs_policy,
    flatten_branched=True, # converte o action_space de MultiDiscrete para Discrete
    single_player=True, # controla um dos jogadores enquanto os outros ficam parados
    opponent_policy=lambda *_: 0,  # faz os oponentes ficarem parados
)

# Obtem tamanhos de estado e ação
state_size = env.observation_space.shape[0]
action_size = env.action_space.n

print("Tamanho do estado: {}, tamanho da ação: {}".format(state_size, action_size))
env.close()

In [ ]:
from ray import tune

def create_rllib_env(env_config: dict = {}):
    # suporte a múltiplas instâncias do ambiente na mesma máquina
    if hasattr(env_config, "worker_index"):
        env_config["worker_id"] = (
            env_config.worker_index * env_config.get("num_envs_per_worker", 1)
            + env_config.vector_index
        )
    return soccer_twos.make(**env_config)

# registra ambiente no Ray
tune.registry.register_env("Soccer", create_rllib_env)

Utilize a configuração abaixo como ponto de partida para seus testes. 

A parte mais imporante é a chave `env_config`, que configura o ambiente para ser compatível com o agente disponibilizado para exportação do seu agente. Neste ponto do curso você já deve conseguir testar as outras variações do ambiente e utilizar as APIs do Ray para treinar um agente próximo (ou melhor) do que o [ceia_baseline_agent](https://drive.google.com/file/d/1WEjr48D7QG9uVy1tf4GJAZTpimHtINzE/view). Exemplos de como utilizar as outras variações podem ser encontrados [aqui](https://github.com/dlb-rl/rl-tournament-starter/). Ao utilizar essas variações, você deve utilizar também outras definições de agente para lidar com os diferentes espaços de observação e ação (que também estão presentes nos exemplos).

In [ ]:
NUM_ENVS_PER_WORKER = 2

analysis = tune.run(
    "PPO",
    config={
        # system settings
        "num_gpus": 1,
        "num_workers": 1,
        "num_envs_per_worker": NUM_ENVS_PER_WORKER,
        "log_level": "INFO",
        "framework": "torch",
        # RL setup
        "env": "Soccer",
        "env_config": {
            "num_envs_per_worker": NUM_ENVS_PER_WORKER,
            "variation": soccer_twos.EnvType.team_vs_policy,
            "single_player": True,
            "flatten_branched": True,
        },
    },
    stop={
        # 10000000 (10M) de steps podem ser necessários para aprender uma política útil
        "timesteps_total": 10000000,
        # você também pode limitar por tempo, de acordo com o tempo limite do colab
        "time_total_s": 14400, # 4h
    },
    checkpoint_freq=100,
    checkpoint_at_end=True,
    local_dir=os.path.join(DRIVE_PATH, "results")
)

## Exportando seu agente treinado

Assim como no Lab 02, você pode exportar seu agente treinado para ser executado como competidor no ambiente da competição ou simplesmente assistí-lo. Para isso, devemos definir uma classe de agente que implemente a interface e trate as observações/ações para o formato da competição. Abaixo, configuramos qual experimento/checkpoint exportar e guardamos a implementação em uma variável para salvá-la em um arquivo posteriormente.

In [ ]:
ALGORITHM = "PPO"
TRIAL = analysis.get_best_logdir("episode_reward_mean", "max")
CHECKPOINT = analysis.get_best_checkpoint(
  TRIAL,
  "training_iteration",
  "max",
)
TRIAL, CHECKPOINT

In [ ]:
agent_file = f"""
import pickle
import os

import gym
from gym_unity.envs import ActionFlattener
import ray
from ray import tune
from ray.tune.registry import get_trainable_cls

from soccer_twos import AgentInterface, DummyEnv


ALGORITHM = "{ALGORITHM}"
CHECKPOINT_PATH = os.path.join(
    os.path.dirname(os.path.abspath(__file__)), 
    "{CHECKPOINT.split("lab03/")[1]}"
)


class MyRaySoccerAgent(AgentInterface):
    def __init__(self, env: gym.Env):
        super().__init__()
        ray.init(ignore_reinit_error=True)

        self.flattener = ActionFlattener(env.action_space.nvec)

        # Load configuration from checkpoint file.
        config_path = ""
        if CHECKPOINT_PATH:
            config_dir = os.path.dirname(CHECKPOINT_PATH)
            config_path = os.path.join(config_dir, "params.pkl")
            # Try parent directory.
            if not os.path.exists(config_path):
                config_path = os.path.join(config_dir, "../params.pkl")

        # Load the config from pickled.
        if os.path.exists(config_path):
            with open(config_path, "rb") as f:
                config = pickle.load(f)
        else:
            # If no config in given checkpoint -> Error.
            raise ValueError(
                "Could not find params.pkl in either the checkpoint dir or "
                "its parent directory!"
            )

        # no need for parallelism on evaluation
        config["num_workers"] = 0
        config["num_gpus"] = 0

        # create a dummy env since it's required but we only care about the policy
        obs_space = env.observation_space
        act_space = self.flattener.action_space
        tune.registry.register_env(
            "DummyEnv",
            lambda *_: DummyEnv(obs_space, act_space),
        )
        config["env"] = "DummyEnv"

        # create the Trainer from config
        cls = get_trainable_cls(ALGORITHM)
        agent = cls(env=config["env"], config=config)
        # load state from checkpoint
        agent.restore(CHECKPOINT_PATH)
        # get default policy for evaluation
        self.policy = agent.get_policy()

    def act(self, observation):
        actions = {{}}
        for player_id in observation:
            # compute_single_action returns a tuple of (action, action_info, ...)
            # as we only need the action, we discard the other elements
            actions[player_id] = self.flattener.lookup_action(
                self.policy.compute_single_action(observation[player_id])[0]
            )
        return actions
"""

In [ ]:
import os
import shutil

agent_name = "my_ray_soccer_agent"
agent_path = os.path.join(DRIVE_PATH, agent_name, agent_name)
shutil.rmtree(agent_path)
os.makedirs(agent_path)

# salva a classe do agente
with open(os.path.join(agent_path, "agent.py"), "w") as f:
    f.write(agent_file)

# salva um __init__ para criar o módulo Python
with open(os.path.join(agent_path, "__init__.py"), "w") as f:
    f.write("from .agent import MyRaySoccerAgent")

# copia o trial inteiro, incluindo os arquivos de configuração do experimento
shutil.copytree(TRIAL, os.path.join(agent_path, TRIAL.split("lab03/")[1]))

# empacota tudo num arquivo .zip
shutil.make_archive(os.path.join(DRIVE_PATH, agent_name), "zip", os.path.join(DRIVE_PATH, agent_name))

Após empacotar todos os arquivos necessários para a execução do seu agente, será criado um arquivo `minicurso_rl/lab03/my_ray_soccer_agent.zip` nos arquivos do Colab e na pasta correspondente no Google Drive. Baixe o arquivo e extraia-o para alguma pasta no seu computador. 

Assumindo que o ambiente Python já está configurado (e.g. os pacotes no [requirements.txt](https://github.com/dlb-rl/rl-tournament-starter/blob/main/requirements.txt) estão instalados), rode `python -m soccer_twos.watch -m my_ray_soccer_agent` para assistir seu agente jogando contra si mesmo. 

Você também pode testar dois agentes diferentes jogando um contra o outro. Utilize o seguinte comando: `python -m soccer_twos.watch -m1 my_ray_soccer_agent -m2 ceia_baseline_agent`. Você pode baixar o agente *ceia_baseline_agent* [aqui](https://drive.google.com/file/d/1WEjr48D7QG9uVy1tf4GJAZTpimHtINzE/view).